In [1]:
%matplotlib inline


Autograd: Automatic Differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with
``requires_grad=True``, but for which we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



In [2]:
import torch

In [3]:
# print all attributes
def DCG_node(listValue):
    for (i, v) in enumerate(listValue):
        print(f"{i}.data: {v.data}\n{i}.requires_grad: {v.requires_grad}\n{i}.grad: {v.grad}\n{i}.grad_fn: {v.grad_fn}\n{i}.is_leaf: {v.is_leaf}\n")


Generate a calculation tree
------------------------
Create a tensor and set ``requires_grad=True`` to track computation with it

In [4]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
l = out * 2

check the Dynamic Computational graph of each node

Here we can see x as an initial point of this calculation series, marked as leaf

In [5]:
DCG_node([x, y, z, out, l])

0.data: tensor([[1., 1.],
        [1., 1.]])
0.requires_grad: True
0.grad: None
0.grad_fn: None
0.is_leaf: True

1.data: tensor([[3., 3.],
        [3., 3.]])
1.requires_grad: True
1.grad: None
1.grad_fn: <AddBackward0 object at 0x7fc47c767c18>
1.is_leaf: False

2.data: tensor([[27., 27.],
        [27., 27.]])
2.requires_grad: True
2.grad: None
2.grad_fn: <MulBackward0 object at 0x7fc47c767be0>
2.is_leaf: False

3.data: 27.0
3.requires_grad: True
3.grad: None
3.grad_fn: <MeanBackward0 object at 0x7fc47c767ba8>
3.is_leaf: False

4.data: 54.0
4.requires_grad: True
4.grad: None
4.grad_fn: <MulBackward0 object at 0x7fc47c767ba8>
4.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
flag in-place. The input flag defaults to ``False`` if not given.



In [6]:
x.requires_grad_(True) # you can't specify y.requires_grad_(False)
DCG_node([x, y, z, out, l])

0.data: tensor([[1., 1.],
        [1., 1.]])
0.requires_grad: True
0.grad: None
0.grad_fn: None
0.is_leaf: True

1.data: tensor([[3., 3.],
        [3., 3.]])
1.requires_grad: True
1.grad: None
1.grad_fn: <AddBackward0 object at 0x7fc47c767cf8>
1.is_leaf: False

2.data: tensor([[27., 27.],
        [27., 27.]])
2.requires_grad: True
2.grad: None
2.grad_fn: <MulBackward0 object at 0x7fc47c767c88>
2.is_leaf: False

3.data: 27.0
3.requires_grad: True
3.grad: None
3.grad_fn: <MeanBackward0 object at 0x7fc47c767d68>
3.is_leaf: False

4.data: 54.0
4.requires_grad: True
4.grad: None
4.grad_fn: <MulBackward0 object at 0x7fc47c767d68>
4.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


Gradients
---------
Let's backprop now.
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(L.))``.



In [7]:
out.backward(retain_graph=True) # δL / δout = 1: L = out + constant

The grad of each node was calculated

In [8]:
DCG_node([x, y, z, out, l])

0.data: tensor([[1., 1.],
        [1., 1.]])
0.requires_grad: True
0.grad: tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
0.grad_fn: None
0.is_leaf: True

1.data: tensor([[3., 3.],
        [3., 3.]])
1.requires_grad: True
1.grad: None
1.grad_fn: <AddBackward0 object at 0x7fc47c767d30>
1.is_leaf: False

2.data: tensor([[27., 27.],
        [27., 27.]])
2.requires_grad: True
2.grad: None
2.grad_fn: <MulBackward0 object at 0x7fc47c767c88>
2.is_leaf: False

3.data: 27.0
3.requires_grad: True
3.grad: None
3.grad_fn: <MeanBackward0 object at 0x7fc47c767cf8>
3.is_leaf: False

4.data: 54.0
4.requires_grad: True
4.grad: None
4.grad_fn: <MulBackward0 object at 0x7fc47c767cf8>
4.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


let's try l.backward()

In [9]:
l.backward(retain_graph=True)
DCG_node([x, y, z, out, l])

0.data: tensor([[1., 1.],
        [1., 1.]])
0.requires_grad: True
0.grad: tensor([[13.5000, 13.5000],
        [13.5000, 13.5000]])
0.grad_fn: None
0.is_leaf: True

1.data: tensor([[3., 3.],
        [3., 3.]])
1.requires_grad: True
1.grad: None
1.grad_fn: <AddBackward0 object at 0x7fc4c6eac400>
1.is_leaf: False

2.data: tensor([[27., 27.],
        [27., 27.]])
2.requires_grad: True
2.grad: None
2.grad_fn: <MulBackward0 object at 0x7fc4c6eac470>
2.is_leaf: False

3.data: 27.0
3.requires_grad: True
3.grad: None
3.grad_fn: <MeanBackward0 object at 0x7fc4c6eac358>
3.is_leaf: False

4.data: 54.0
4.requires_grad: True
4.grad: None
4.grad_fn: <MulBackward0 object at 0x7fc4c6eac358>
4.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


the expected result of x.grad is 9 but actually, and unfortunately, x.grad is 9 + 4.5(the result when we calculate out.backward()). so we need to set the grad.data to zero

In [ ]:
x.grad.data.zero_()
l.backward(retain_graph=True)
DCG_node([x, y, z, out, l])

0.data: tensor([[1., 1.],
        [1., 1.]])
0.requires_grad: True
0.grad: tensor([[9., 9.],
        [9., 9.]])
0.grad_fn: None
0.is_leaf: True

1.data: tensor([[3., 3.],
        [3., 3.]])
1.requires_grad: True
1.grad: None
1.grad_fn: <AddBackward0 object at 0x7f32c41d8160>
1.is_leaf: False

2.data: tensor([[27., 27.],
        [27., 27.]])
2.requires_grad: True
2.grad: None
2.grad_fn: <MulBackward0 object at 0x7f32c41d8c88>
2.is_leaf: False

3.data: 27.0
3.requires_grad: True
3.grad: None
3.grad_fn: <MeanBackward0 object at 0x7f32c41d8a58>
3.is_leaf: False

4.data: 54.0
4.requires_grad: True
4.grad: None
4.grad_fn: <MulBackward0 object at 0x7f32c41d8a58>
4.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


what if we use y.backward()? error raise: grad can be implicitly created only for scalar outputs

In [ ]:
y.backward()
DCG_node([x, y, z, out, l])

RuntimeError: ignored

You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



Mathematically, if you have a vector valued function $\vec{y}=f(\vec{x})$,
then the gradient of $\vec{y}$ with respect to $\vec{x}$
is a Jacobian matrix:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
compute the product $v^{T}\cdot J$. If $v$ happens to be
the gradient of a scalar function $l=g\left(\vec{y}\right)$,
that is,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(Note that $v^{T}\cdot J$ gives a row vector which can be
treated as a column vector by taking $J^{T}\cdot v$.)

This characteristic of vector-Jacobian product makes it very
convenient to feed external gradients into a model that has
non-scalar output.



Now let's take a look at an example of vector-Jacobian product:



In [ ]:
x = torch.randn(3, requires_grad=True)
print(x)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y/x)

tensor([-2.0547,  0.4526,  0.5030], requires_grad=True)
tensor([512., 512., 512.], grad_fn=<DivBackward0>)


In [ ]:
DCG_node([x, y])

0.data: tensor([-2.0547,  0.4526,  0.5030])
0.requires_grad: True
0.grad: None
0.grad_fn: None
0.is_leaf: True

1.data: tensor([-1052.0129,   231.7180,   257.5281])
1.requires_grad: True
1.grad: None
1.grad_fn: <MulBackward0 object at 0x7f32bbd510f0>
1.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


Now in this case ``y`` is no longer a scalar. ``torch.autograd``
could not compute the full Jacobian directly, but if we just
want the vector-Jacobian product, simply pass the vector to
``backward`` as argument:



In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
# v = dl / dy, so y.backward(v) mean the supreme is l, and grad has been calculated and populated
# pay attention, l is a scalar value
# v1 = dl / dy1, v2 = dl / dy2, v3 = dl / dy3
# l = y1 * v1 + y2 * v2 + y3 * v3
y.backward(v)

In [ ]:
DCG_node([x, y, v])

0.data: tensor([-2.0547,  0.4526,  0.5030])
0.requires_grad: True
0.grad: tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])
0.grad_fn: None
0.is_leaf: True

1.data: tensor([-1052.0129,   231.7180,   257.5281])
1.requires_grad: True
1.grad: None
1.grad_fn: <MulBackward0 object at 0x7f32bbd51630>
1.is_leaf: False

2.data: tensor([1.0000e-01, 1.0000e+00, 1.0000e-04])
2.requires_grad: False
2.grad: None
2.grad_fn: None
2.is_leaf: True



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


In [ ]:
print(x.grad) # x.grad = dl / dx

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` either by wrapping the code block in
``with torch.no_grad():``



In [ ]:
print(x.requires_grad)
t = x ** 2
print((x ** 2).requires_grad)
print(t.requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)
	print(t.requires_grad)
	
DCG_node([x, t])

True
True
True
False
True
0.data: tensor([-2.0547,  0.4526,  0.5030])
0.requires_grad: True
0.grad: tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])
0.grad_fn: None
0.is_leaf: True

1.data: tensor([4.2218, 0.2048, 0.2530])
1.requires_grad: True
1.grad: None
1.grad_fn: <PowBackward0 object at 0x7f32bbd03320>
1.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


Or by using ``.detach()`` to get a new Tensor with the same
content but that does not require gradients:



In [ ]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())
print(x)
print(y)

True
False
tensor(True)
tensor([-2.0547,  0.4526,  0.5030], requires_grad=True)
tensor([-2.0547,  0.4526,  0.5030])


## grad_fn

In [ ]:
import torch

In [ ]:
x1 = torch.tensor([1, 0.1, 0.09, 0.99], requires_grad=True)
x2 = x1 + 3
x3 = x2 ** 2
x4 = sum(x3)
print(x4.grad_fn)
print(x3.grad_fn)
print(x2.grad_fn)
print(x1.grad_fn)

None


In [ ]:
print(x4.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0])

in the function
------------------------
create a function to test it

In [ ]:
x = torch.tensor(torch.FloatTensor([[1, 2], [3, 4]]), requires_grad=True)
y = torch.sum(x ** 2)
y.backward()
DCG_node([x, y])

0.data: tensor([[1., 2.],
        [3., 4.]])
0.requires_grad: True
0.grad: tensor([[2., 4.],
        [6., 8.]])
0.grad_fn: None
0.is_leaf: True

1.data: 30.0
1.requires_grad: True
1.grad: None
1.grad_fn: <SumBackward0 object at 0x7f32bbd03eb8>
1.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


In [ ]:
x = torch.tensor(torch.FloatTensor([[1, 2], [3, 4]]), requires_grad=True)
def sum_square(x):
    y = torch.sum(x ** 3)
    u = y * 2
    return u
z = sum_square(x)
z.backward()
DCG_node([x, z])


0.data: tensor([[1., 2.],
        [3., 4.]])
0.requires_grad: True
0.grad: tensor([[ 6., 24.],
        [54., 96.]])
0.grad_fn: None
0.is_leaf: True

1.data: 200.0
1.requires_grad: True
1.grad: None
1.grad_fn: <MulBackward0 object at 0x7f32bbd51f98>
1.is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


**Read Later:**

Document about ``autograd.Function`` is at
https://pytorch.org/docs/stable/autograd.html#function

